In [5]:
import pandas as pd

orders = pd.read_csv('/content/orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [6]:
users = pd.read_json('/content/users.json')
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [7]:
import sqlite3

conn = sqlite3.connect('food.db')
cur = conn.cursor()
with open('/content/restaurants.sql', 'r') as f:
    cur.executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants;", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [8]:
merged_1 = pd.merge(
    orders,
    users,
    how='left',
    left_on='user_id',
    right_on='user_id'
)
final_df = pd.merge(
    merged_1,
    restaurants,
    how='left',
    left_on='restaurant_id',
    right_on='restaurant_id'
)
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

In [10]:
import pandas as pd
import sqlite3
df = pd.read_csv('/content/final_food_delivery_dataset.csv')
conn = sqlite3.connect('food_analysis.db')
df.to_sql('food_data', conn, if_exists='replace', index=False)

10000

In [11]:
pd.read_sql("SELECT * FROM food_data LIMIT 5;", conn)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [13]:
pd.read_sql("""
SELECT city, SUM(total_amount) AS gold_revenue
FROM food_data
WHERE membership = 'Gold'
GROUP BY city
ORDER BY gold_revenue DESC
LIMIT 1;
""", conn)

,city,gold_revenue
0,Chennai,1080909.79


In [15]:
pd.read_sql("""
SELECT
strftime('%m', order_date) AS month,
COUNT(*) AS total_orders
FROM food_data
GROUP BY month
ORDER BY month;
""", conn)

,month,total_orders
0,None,10000


In [16]:
pd.read_sql("""
SELECT
user_id,
SUM(total_amount) AS total_spent
FROM food_data
GROUP BY user_id
ORDER BY total_spent DESC
LIMIT 5;
""", conn)

,user_id,total_spent
0,1515,11556.49
1,650,10747.44
2,496,9634.30
3,2586,9486.61
4,2615,9237.32


In [17]:
pd.read_sql("""
SELECT
    cuisine,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY cuisine
ORDER BY revenue DESC
LIMIT 1;
""", conn)

,cuisine,revenue
0,Mexican,2085503.09


In [18]:
pd.read_sql("""
SELECT
    city,
    COUNT(*) AS orders
FROM food_data
GROUP BY city
ORDER BY orders DESC;
""", conn)

,city,orders
0,Bangalore,2751
1,Chennai,2469
2,Pune,2430
3,Hyderabad,2350


In [21]:
pd.read_sql("""
SELECT
    membership,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY membership;
""", conn)

,membership,revenue
0,Gold,3975364.89
1,Regular,4036259.23


In [22]:
pd.read_sql("""
SELECT
    strftime('%m', order_date) AS month,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY month
ORDER BY revenue DESC
LIMIT 1;
""", conn)

,month,revenue
0,None,8011624.12


In [21]:
final_df.dtypes

,0
order_id,int64
user_id,int64
restaurant_id,int64
order_date,datetime64[ns]
total_amount,float64
restaurant_name_x,object
name,object
city,object
membership,object
restaurant_name_y,object


In [22]:
pd.read_sql("SELECT COUNT(*) FROM food_data;", conn)

,COUNT(*)
0,10000


In [23]:
pd.read_sql("""
SELECT cuisine, AVG(total_amount) AS avg_order_value
FROM food_data
GROUP BY cuisine
ORDER BY avg_order_value DESC
LIMIT 1; """, conn)

,cuisine,avg_order_value
0,Mexican,808.021344


In [24]:
pd.read_sql("""
SELECT COUNT(*) AS users_above_1000
FROM (
    SELECT user_id, SUM(total_amount) AS total_spent
    FROM food_data
    GROUP BY user_id
    HAVING total_spent > 1000
); """, conn)

,users_above_1000
0,2544


In [25]:
pd.read_sql("""
SELECT
    CASE
        WHEN rating >= 4.5 THEN '4.5-5.0'
        WHEN rating >= 4.0 THEN '4.0-4.5'
        WHEN rating >= 3.5 THEN '3.5-4.0'
        ELSE 'Below 3.5'
    END AS rating_range,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY rating_range
ORDER BY revenue DESC
LIMIT 1; """, conn)

,rating_range,revenue
0,4.5-5.0,2676077.78


In [27]:
pd.read_sql("""
SELECT city, AVG(total_amount) AS avg_value
FROM food_data
WHERE membership = 'Gold'
GROUP BY city
ORDER BY avg_value DESC
LIMIT 1; """, conn)

,city,avg_value
0,Chennai,808.45908


In [30]:
pd.read_sql("""
SELECT
    cuisine,
    COUNT(DISTINCT restaurant_id) AS restaurant_count,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY cuisine
ORDER BY restaurant_count ASC, revenue DESC
LIMIT 1; """, conn)

,cuisine,restaurant_count,revenue
0,Chinese,120,1930504.65


In [32]:
pd.read_sql("""
SELECT
    ROUND(
        100.0 * SUM(CASE WHEN membership='Gold' THEN 1 ELSE 0 END)
        / COUNT(*)
    ) AS gold_order_percentage
FROM food_data;
 """, conn)

,gold_order_percentage
0,50.0


In [35]:
pd.read_sql("""
SELECT
    restaurant_name_y,
    AVG(total_amount) AS avg_value,
    COUNT(*) AS orders
FROM food_data
GROUP BY restaurant_name_y
HAVING orders < 20
ORDER BY avg_value DESC
LIMIT 1;
 """, conn)

,restaurant_name_y,avg_value,orders
0,Restaurant_294,1040.222308,13


In [36]:
pd.read_sql("""
SELECT
    city,
    cuisine,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY city, cuisine
ORDER BY revenue DESC
LIMIT 1;
 """, conn)

,city,cuisine,revenue
0,Bangalore,Mexican,571004.61


In [37]:
pd.read_sql("""
SELECT
    CASE
        WHEN strftime('%m', order_date) IN ('01','02','03') THEN 'Q1'
        WHEN strftime('%m', order_date) IN ('04','05','06') THEN 'Q2'
        WHEN strftime('%m', order_date) IN ('07','08','09') THEN 'Q3'
        ELSE 'Q4'
    END AS quarter,
    SUM(total_amount) AS revenue
FROM food_data
GROUP BY quarter
ORDER BY revenue DESC
LIMIT 1;
 """, conn)

,quarter,revenue
0,Q4,8011624.12


In [38]:
pd.read_sql(""" SELECT COUNT(*) AS total_gold_orders
FROM food_data
WHERE membership = 'Gold'; """, conn)

,total_gold_orders
0,4987


In [39]:
pd.read_sql(""" SELECT ROUND(SUM(total_amount)) AS hyderabad_revenue
FROM food_data
WHERE city = 'Hyderabad'; """, conn)

,hyderabad_revenue
0,1889367.0


In [40]:
pd.read_sql(""" SELECT COUNT(DISTINCT user_id) AS distinct_users
FROM food_data; """, conn)

,distinct_users
0,2883


In [41]:
pd.read_sql("""SELECT ROUND(AVG(total_amount), 2) AS avg_gold_order_value
FROM food_data
WHERE membership = 'Gold';""", conn)

,avg_gold_order_value
0,797.15


In [42]:
pd.read_sql(""" SELECT COUNT(*) AS high_rating_orders
FROM food_data
WHERE rating >= 4.5; """, conn)

,high_rating_orders
0,3374


In [43]:
pd.read_sql(""" SELECT COUNT(*) AS orders_in_top_gold_city
FROM food_data
WHERE membership = 'Gold'
AND city = (
    SELECT city
    FROM food_data
    WHERE membership = 'Gold'
    GROUP BY city
    ORDER BY SUM(total_amount) DESC
    LIMIT 1
); """, conn)

,orders_in_top_gold_city
0,1337


In [ ]:
pd.read_sql("""  """, conn)

In [ ]:
pd.read_sql("""  """, conn)